### Correction of the ML-Ready Cell-Painting Data Sets
#### Prerequisite
This notebook is part of the master thesis of Luis Vollmers. It uses the cell paintin data set of Bray et al. and the goal of the project is to predict pubchem assay data. Originally this notebook was concerned with pubchem AID 1030 but this might be subject to change.
#### Introduction
The datasets generated in ml_ready of step 01 comprise several mistakes that have been made and that need to be corrected within this jupyter notebook. Mostly, the compounds listed in the rows of the ML-ready dataframes are not unique, due to a misunderstanding stemming from the Bray et al Cell-Painting paper. It was stated that the Metadata_broad_sample is a unique identifier which was found to be wrong. Identical compounds which were used in different concentrations and/or on different well-plates were assigned different Metadata_broad_sample values. This needs to be corrected as well as the averaging over the different column values. In the data columns, the cell painting features are listed and so far the algorithm just kept the first concentration and took the average of the respective multiplicates. The correct way of doing it however is to check how many concentrations are present and then take the median of the concentration that is most frequent which is done by this jupyter notebook. 
#### Summary of the steps in this notebook
1. Import the ML-ready pubchem-assay and the preprocessed cell-painting raw data
2. Reintroduce the center median data into the pubchem df
3. Treat the multi-concentration Compounds adequately
4. Export the output


In [1]:
import pandas as pd

#### 1. Import the ML-ready pubchem-assay and the preprocessed cell-painting raw data
- Inputs are taken from the directory for step 1 of the pubchem assays and from the preprocessing directory
- the data from cp_1030 is erroneous but the meta data is needed for the overlap with the center median data
- therefore filter the df for the metadata

In [16]:
# define the meta cols of the pubchem assays that are relevant and filter the df 
meta_cols = ['Metadata_broad_sample','PUBCHEM_ACTIVITY_OUTCOME', 'PUBCHEM_ACTIVITY_SCORE']

# load the data into RAM 'cp_1030' can be used as a variable for a bash script
dataset_path='../../01-FilteringAssays/ml_ready/cp_651744.csv'
df = pd.read_csv(dataset_path, usecols=meta_cols)

# read the cell-painting data into RAM
cp = pd.read_csv('../../../preprocessing/cp_center_median.csv', index_col=0)

,Metadata_broad_sample,PUBCHEM_ACTIVITY_OUTCOME,PUBCHEM_ACTIVITY_SCORE
0,BRD-A98248982-001-08-0,Inactive,1.0
1,BRD-K00124439-001-01-2,Inactive,4.0
2,BRD-K00328666-001-02-1,Active,76.0
3,BRD-K00350773-001-01-7,Active,77.0
4,BRD-K00468432-001-02-8,Inactive,7.0
...,...,...,...
499,BRD-M36167927-001-01-6,Inactive,13.0
500,BRD-M49151292-001-01-2,Active,52.0
501,BRD-M76969896-001-01-3,Inactive,0.0
502,BRD-M78319284-001-01-9,Inactive,-1.0


#### 2. Reintroduce the Center Median Data into the Pubchem DF
- the common column is Metadata_broad_sample which was responsible for the errors in the first place
- it is actually the only identifier column that is left in the ML-ready df
- the inner merge makes sure that only rows are kept that can be found in both dataframes

In [3]:
# merge the cp and the assay data on the common column
merged = pd.merge(left=cp, right=df, on='Metadata_broad_sample', how='inner')

In [4]:
# this cell calculates the number of rows expected from the final dataframe
multi_list = []
concs_list = merged.CAN_SMILES.value_counts().to_list()

for i in concs_list:
    if i > 1:
        multi_list.append(i)
        
merged.shape[0]-sum(multi_list)+len(multi_list)

493

#### 3. Treat the multi-concentration Compounds adequately
- this step is the only one that needs to be conducted manually
- first look at the multi conc compounds and determine how to treat them
- in this case the concentrations that are less frequent are dropped and the remaining ones are calculated into median
- for that purpose the merged DF is split into data and meta columns and only the data columns are treated accordingly
- the groupby method is used to calculate the compound wise median 
- the single concentrations are already calculated so only the multi concs are actually computed herein

In [5]:
# this cell checks hoif 2 different concentrations are present per compound
# sees if the two concs differ only alittle bit and writes the compounds which are highly differing into alist
# it also writes the more frequent concentration into that same list
multi_merged = merged.query('SINGLE_CONC==False').loc[:,['CAN_SMILES','Metadata_mmoles_per_liter']]

len_list = []
compound_list = []

for i,j in multi_merged.groupby("CAN_SMILES"):
    len_list.append(j.Metadata_mmoles_per_liter.value_counts().shape[0])
    
if all(flag == 2 for flag in len_list):
    for i,j in multi_merged.groupby("CAN_SMILES"):
        avr = (j.Metadata_mmoles_per_liter.value_counts().index.to_list()[0] + j.Metadata_mmoles_per_liter.value_counts().index.to_list()[1])/2
        dev = abs(avr - j.Metadata_mmoles_per_liter.value_counts().index.to_list()[1])
        if dev/avr > 0.1:
            compound_list.append([i,j.Metadata_mmoles_per_liter.value_counts().index.to_list()[0]])
else:
    print("higher doubly concs or all singly! ")
    
compound_list

[]

In [6]:
for compound in compound_list:
    merged = merged.drop(merged[(merged.Metadata_mmoles_per_liter != compound[1]) & (merged.CAN_SMILES==compound[0])].index)

In [7]:
# Quality Control step that makes sure only the relevant rows with the most concentrations are kept
multi_merged = merged.query('SINGLE_CONC==False').loc[:,['CAN_SMILES','Metadata_mmoles_per_liter']]
for i,j in multi_merged.groupby("CAN_SMILES"):
    print("{}\n{}\n\n".format(i,j.Metadata_mmoles_per_liter.value_counts()))

CC(C)COC(=O)N(C)C[C@@H]1OCc2cnnn2CCCC(=O)N([C@H](C)CO)C[C@H]1C
5.008734    4
4.837163    4
Name: Metadata_mmoles_per_liter, dtype: int64


CN1C(=O)c2cc(NC(=O)c3cccc(F)c3)ccc2OC[C@@H]2O[C@H](CC(=O)NCC3CCCCC3)CC[C@@H]21
5.001511    8
4.996119    8
Name: Metadata_mmoles_per_liter, dtype: int64


C[C@@H](CO)N1C[C@@H](C)[C@H](CN(C)C(=O)C2CC2)OCc2cn(nn2)CCCC1=O
5.005668    4
5.002573    4
Name: Metadata_mmoles_per_liter, dtype: int64


C[C@@H](CO)N1C[C@@H](C)[C@H](CN(C)C(=O)c2ccc(N3CCN(C)CC3)cc2)OCc2cn(nn2)CCCC1=O
4.878387    4
5.002223    4
Name: Metadata_mmoles_per_liter, dtype: int64


C[C@@H](CO)N1C[C@@H](C)[C@H](CN(C)S(=O)(=O)c2ccc3ccccc3c2)OCc2cn(nn2)CCCC1=O
5.066968    4
5.017491    4
Name: Metadata_mmoles_per_liter, dtype: int64


C[C@@H]1CN([C@H](C)CO)C(=O)CCCn2cc(nn2)CO[C@H]1CN(C)C(=O)c1ccc2ccn(C)c2c1
5.233043    4
5.016112    4
Name: Metadata_mmoles_per_liter, dtype: int64


C[C@H]1CN([C@@H](C)CO)C(=O)CCCn2cc(nn2)CO[C@@H]1CN(C)C(=O)Cc1ccccc1
5.002088    4
5.189670    4
Name: Metad

In [8]:
# define a list of the all columns of the merged data frame
all_cols = merged.columns.to_list()
# redefine meta columns with the meta information of the cell painting assay
meta_cols = ['CAN_SMILES','CPD_SMILES','Metadata_broad_sample','Metadata_Plate_Map_Name','Metadata_ASSAY_WELL_ROLE','Metadata_Plate','SINGLE_CONC','PUBCHEM_ACTIVITY_SCORE','PUBCHEM_ACTIVITY_OUTCOME']

In [9]:
# data cols are basically all columns without the meta data. hence the forloop that removes those from data_cols
data_cols = all_cols
for item in meta_cols:
    data_cols.remove(item)
    
# afterwards the 'CAN_SMILES' column is inserted at the first position
data_cols.insert(0,'CAN_SMILES')

In [10]:
# the merged data frame is split into two dataframes containing meta and raw data information
merged_data = merged.loc[:,data_cols]
merged_meta = merged.loc[:,meta_cols]

In [11]:
# this command takes the compound wise median of the data
merged_data = merged_data.groupby('CAN_SMILES').median().reset_index()

#### 4. Export the Output
- as a last step the only thing that needs to be done is to merge the meta and data columns back into one DF
- a bit of a clean up needs to be done since the merge command creates suplicates, which can be safely deleted
- output in csv format named according to the pubchem AID

In [12]:
# the median data is merged back with the meta data
merged = pd.merge(left=merged_meta, right=merged_data, on='CAN_SMILES', how='left')

In [13]:
# merging generally keeps all rows in both frames so that duplicates are generated, which get hereby deleted
merged = merged.drop_duplicates(subset='CAN_SMILES')

In [14]:
merged.to_csv('../_output/cp_651744.csv',index=False)

In [15]:
pd.read_csv('../_output/cp_651744.csv') # only uncomment for quality control purposes, i.e. visual conformation

,CAN_SMILES,CPD_SMILES,Metadata_broad_sample,Metadata_Plate_Map_Name,Metadata_ASSAY_WELL_ROLE,Metadata_Plate,SINGLE_CONC,PUBCHEM_ACTIVITY_SCORE,PUBCHEM_ACTIVITY_OUTCOME,Metadata_mmoles_per_liter,...,Nuclei_Texture_Variance_DNA_5_0,Nuclei_Texture_Variance_ER_10_0,Nuclei_Texture_Variance_ER_3_0,Nuclei_Texture_Variance_ER_5_0,Nuclei_Texture_Variance_Mito_10_0,Nuclei_Texture_Variance_Mito_3_0,Nuclei_Texture_Variance_Mito_5_0,Nuclei_Texture_Variance_RNA_10_0,Nuclei_Texture_Variance_RNA_3_0,Nuclei_Texture_Variance_RNA_5_0
0,C[C@H]1CN([C@@H](C)CO)C(=O)c2cc(NC(=O)Nc3cccc4...,C[C@@H](CO)N1C[C@H](C)[C@H](CN(C)Cc2ccc3OCOc3c...,BRD-K76922696-001-01-6,H-CBLA-001-4,treated,24507,True,29.0,Active,4.960596,...,0.063016,-0.016443,-0.006653,-0.015067,0.010266,-0.001593,-0.006010,-0.015627,0.012799,0.014589
1,COc1ccc(NC(=O)Nc2ccc3c(c2)CC(=O)N([C@H](C)CO)C...,COc1ccc(NC(=O)Nc2ccc3O[C@@H](CN(C)CC4CCCCC4)[C...,BRD-K32577562-001-01-2,H-CBLA-001-4,treated,24507,True,105.0,Active,4.953589,...,0.097017,0.112506,0.071561,0.099244,-0.003877,-0.067278,-0.062642,0.025726,0.032104,0.036542
2,C[C@H]1CN([C@H](C)CO)C(=O)Cc2cc(N(C)C)ccc2O[C@...,C[C@H](CO)N1C[C@H](C)[C@@H](CN(C)Cc2ccc(cc2)C(...,BRD-K00808919-001-01-9,H-CBLA-001-4,treated,24507,True,77.0,Active,4.944559,...,-0.105841,-0.065828,-0.076355,-0.078330,-0.132142,-0.172822,-0.168981,0.104771,0.061807,0.078668
3,C[C@@H](CO)N1C[C@@H](C)[C@H](CN(C)Cc2ccc(-c3cc...,C[C@@H](CO)N1C[C@@H](C)[C@H](CN(C)Cc2ccc(cc2)-...,BRD-K85112460-001-01-7,H-CBLA-001-4,treated,24507,True,103.0,Active,4.993321,...,-0.036176,0.077308,0.057949,0.069354,-0.111606,-0.227159,-0.203055,-0.023570,-0.023871,-0.013733
4,C[C@H]1CN([C@H](C)CO)C(=O)c2cc(NC(=O)NC3CCCCC3...,C[C@H](CO)N1C[C@H](C)[C@H](CN(C)Cc2ccc3OCOc3c2...,BRD-K40221630-001-01-5,H-CBLA-001-4,treated,24507,True,57.0,Active,5.007604,...,-0.114902,-0.127187,-0.094253,-0.098546,-0.095863,-0.112678,-0.104231,-0.049647,-0.036560,-0.035651
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
488,CC(C)COC(=O)N(C)C[C@@H]1OCc2cnnn2CCCC(=O)N([C@...,CC(C)COC(=O)N(C)C[C@@H]1OCc2cnnn2CCCC(=O)N(C[C...,BRD-K70102743-001-02-2,H-CBLD-001-4,treated,24726,False,7.0,Inactive,4.922948,...,-0.071412,0.000640,-0.010633,-0.003033,-0.063058,-0.089731,-0.081761,-0.020253,-0.016098,-0.015260
489,C[C@H]1CN([C@H](C)CO)C(=O)CCCn2nncc2CO[C@@H]1C...,C[C@H](CO)N1C[C@H](C)[C@@H](CN(C)C(=O)c2ccc3cc...,BRD-K45656768-001-02-9,H-CBLD-001-4,treated,24726,False,-2.0,Inactive,5.031932,...,-0.055552,-0.050326,-0.062142,-0.044513,-0.055773,-0.066489,-0.082382,-0.045626,-0.060108,-0.061499
490,C[C@@H]1CN([C@H](C)CO)C(=O)CCCn2cc(nn2)CO[C@H]...,C[C@H](CO)N1C[C@@H](C)[C@H](CN(C)C(=O)c2ccc3cc...,BRD-K47380281-001-02-2,H-CBLD-001-4,treated,24726,False,-7.0,Inactive,5.124577,...,-0.020199,0.020621,-0.024418,-0.013695,-0.055857,-0.069990,-0.056783,-0.052381,-0.072124,-0.046940
491,C[C@@H](CO)N1C[C@@H](C)[C@H](CN(C)C(=O)c2ccc(N...,C[C@@H](CO)N1C[C@@H](C)[C@H](CN(C)C(=O)c2ccc(c...,BRD-K17360472-001-02-9,H-CBLD-001-4,treated,24726,False,-7.0,Inactive,4.940305,...,0.016650,0.002742,-0.000302,0.015454,0.004820,-0.010673,-0.044359,-0.072973,-0.063429,-0.067247
